<a href="https://colab.research.google.com/github/inesclopes/data-eng-proc/blob/main/spark_streaming/challenges/final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [ ]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
    - extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [2]:
%pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.2 MB/s eta 0:00:00


In [1]:
%rm -rf content/lake/

# Producer

In [2]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("content/lake/bronze/messages/data")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.outputMode('append')
.option('checkpointLocation', 'content/checkpoint')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_messages)
.start()
)


query.awaitTermination(60)
query.stop() # needed because the line above will stop the running cell but process still continue in the background


# Additional datasets

In [6]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

# Streaming Messages x Messages Corrupted

In [ ]:

corrupted_filters = (F.col("event_type").isNull() |
         (F.col("event_type") == '')   |
          F.isnan("event_type")        |
          F.upper(F.col("event_type")).contains("NONE"))

def handle_messages(df: DataFrame, batch_id):
  base_df = df.join(countries, "country_id", how = "inner").withColumn("date", F.to_date("timestamp"))

  corrupt_df = base_df.filter(corrupted_filters)
  corrupt_df.write.mode("append").format("parquet").partitionBy("date").save("content/lake/silver/messages_corrupted/data")

  clean_df = base_df.filter(corrupted_filters == False)
  clean_df.write.mode("append").format("parquet").partitionBy('date').save("content/lake/silver/messages/data")

False

## Checking data

In [ ]:
message_schema = StructType([StructField('timestamp', TimestampType(), True),
                                StructField('value', LongType(), True),
                                StructField('event_type', StringType(), True),
                                StructField('message_id', StringType(), True),
                                StructField('channel', StringType(), True),
                                StructField('country_id', IntegerType(), True),
                              StructField('user_id', IntegerType(), True) ]
                            )

In [ ]:
df_messages = spark.readStream.format("parquet").option("maxFilesPerTrigger", 1).schema(message_schema).load("content/lake/bronze/messages/data")

# write stream
query = (df_messages.writeStream
.outputMode('append')
.option('checkpointLocation', 'content/lake/silver/messages_corrupted/checkpoint')
.trigger(processingTime='5 seconds')
.foreachBatch(handle_messages)
.start()
)
query.awaitTermination(20)

## Testing stream

In [ ]:
import os
import time

def run_assertion_with_timeout():
    timeout = 5 * 60
    interval = 4
    start_time = time.time()

    while time.time() - start_time < timeout:
      corrupted = os.listdir("content/lake/silver/messages_corrupted/data/date=2024-12-17")
      clean = os.listdir("content/lake/silver/messages/data/date=2024-12-17")
      source = os.listdir("content/lake/bronze/messages/data")
      try:
        assert len(corrupted) + len(clean) == len(source), f"Not all files have been streamed. Silver: {len(corrupted) + len(clean)} and bronze has: {len(source)}"
        print("Assertion succeeded.")
        query.stop()
        return
      except AssertionError as e:
        print(e)

      time.sleep(interval)

    print("Timeout reached. Exiting.")
    query.stop()

run_assertion_with_timeout()


# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

In [38]:
# dedup data
from pyspark.sql import functions as F
from pyspark.sql.window import Window
df = spark.read.format("parquet").load("content/lake/silver/messages")
dedup = df.withColumn("row_number", F.row_number().over(Window.partitionBy("message_id", "event_type", "channel").orderBy("timestamp"))).filter("row_number = 1").drop("row_number")

(1449, 39)

### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [76]:

(dedup
 .groupBy("date", "channel")
 .pivot("event_type")
 .count()
 .show())

+----------+-------+-------+----+--------+----+
|      date|channel|CLICKED|OPEN|RECEIVED|SENT|
+----------+-------+-------+----+--------+----+
|2024-12-17|   PUSH|      2|   4|       4|   2|
|2024-12-17|  EMAIL|      1|   1|       3|NULL|
|2024-12-17|   CHAT|      3|   3|       1|   2|
|2024-12-17|    SMS|      1|   3|       1|   2|
|2024-12-17|  OTHER|      2|   3|    NULL|   1|
+----------+-------+-------+----+--------+----+



## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [108]:
unique_channels = [row['channel'] for row in dedup.select("channel").distinct().collect()]


In [107]:

(dedup
 .groupBy("user_id")
 .pivot("channel")
 .count()
 .fillna(0)
 .withColumn("iterations", sum([F.col(name) for name in unique_channels]))
 .orderBy("iterations", ascending = False)
 .show(5))

+-------+----+-----+-----+----+---+----------+
|user_id|CHAT|EMAIL|OTHER|PUSH|SMS|iterations|
+-------+----+-----+-----+----+---+----------+
|   1005|   1|    0|    0|   0|  2|         3|
|   1019|   1|    0|    0|   1|  1|         3|
|   1026|   1|    0|    2|   0|  0|         3|
|   1031|   0|    0|    1|   1|  0|         2|
|   1028|   0|    0|    0|   2|  0|         2|
+-------+----+-----+-----+----+---+----------+
only showing top 5 rows



In [103]:
sum([1,2,3])

6

# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

- Which storage would you use and why? (database?, data lake?, kafka?)

